In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset
from sklearn.model_selection import train_test_split, KFold
from PIL import Image
import shutil
import pandas
from torchvision.io import read_image

cudnn.benchmark = True
plt.ion()   # interactive mode
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [ ]:
test_size = 0.25

dataset = np.load('data_train.npy').reshape((300, 300, 9032))
labels = np.load('t_train_corrected.npy')
if os.path.exists("train"):
    shutil.rmtree("train")
if os.path.exists("test"):
    shutil.rmtree("test")
os.mkdir("train")
os.mkdir("test")
os.mkdir("train/images")
os.mkdir("train/labels")
os.mkdir("test/images")
os.mkdir("test/labels")

train_indices, test_indices = train_test_split(range(9032), test_size=test_size, shuffle=False)

for i in range(10):
    for idx in np.where(labels[train_indices] == i)[0]:
        image = Image.fromarray(dataset[:, :, idx], mode = 'L').resize((320, 320))
        image.save("train/images/" + str(i) + "_" + str(idx) + ".jpg")
        with open("train/labels/" + str(i) + "_" + str(idx) + ".txt", "w") as f:
            f.write(str(i) + " 0.5 0.5 1 1")
    for idx in np.where(labels[test_indices] == i)[0]:
        image = Image.fromarray(dataset[:, :, idx + len(train_indices)], mode = 'L').resize((320, 320))
        image.save("test/images/" + str(i) + "_" + str(idx + len(train_indices)) + ".jpg")
        with open("test/labels/" + str(i) + "_" + str(idx + len(train_indices)) + ".txt", "w") as f:
            f.write(str(i) + " 0.5 0.5 1 1")
            
for idx in np.where(labels[train_indices] == -1)[0]:
    image = Image.fromarray(dataset[:, :, idx], mode = 'L').resize((320, 320))
    image.save("train/images/" + str(10) + "_" + str(idx) + ".jpg")
    with open("train/labels/" + str(10) + "_" + str(idx) + ".txt", "w") as f:
        f.write(str(10) + " 0.5 0.5 1 1")
for idx in np.where(labels[test_indices] == -1)[0]:
    image = Image.fromarray(dataset[:, :, idx + len(train_indices)], mode = 'L').resize((320, 320))
    image.save("test/images/" + str(10) + "_" + str(idx + len(train_indices)) + ".jpg")
    with open("test/labels/" + str(10) + "_" + str(idx + len(train_indices)) + ".txt", "w") as f:
        f.write(str(10) + " 0.5 0.5 1 1")

In [2]:
def test_model(model, img_root, img_dir, img_labels):
    since = time.time()

    # Each epoch has a training and validation phase
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for idx in range(len(test_labels)):
        img_path = os.path.join(img_root, img_dir[idx])
        image = Image.open(img_path)
        label = img_labels[idx]

        # zero the parameter gradients
        #optimizer.zero_grad()

        # forward
        output = model(image)
        print(output.pandas().xyxy[0].sort_values('xmin'))
        #output.print()
        #_, preds = torch.max(outputs, 1)
        #print(preds)
        #loss = criterion(output, label)

        # statistics
        #running_loss += loss.item() * inputs.size(0)
        #running_corrects += torch.sum(preds == labels.data)

    #loss = running_loss / dataset_test_size
    #acc = running_corrects.double() / dataset_test_size

    #print(f'Loss: {loss:.4f} Acc: {acc:.4f}')

    time_elapsed = time.time() - since
    print(f'Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [4]:
test_root = "test/images"
test_imgs = os.listdir(test_root)
test_labels = np.array([int(img[0]) if img[1] != '_' else 10 for img in os.listdir(test_root)])

model_ft = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\justin.rossiter/.cache\torch\hub\master.zip
requirements: YOLOv5 requirements "gitpython" "tqdm>=4.64.0" not found, attempting AutoUpdate...
requirements:  Command 'pip install "gitpython" "tqdm>=4.64.0"  ' returned non-zero exit status 1.
YOLOv5  2022-11-29 Python-3.8.8 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients
Adding AutoShape... 


In [5]:
test_model(model_ft, test_root, test_imgs, test_labels)

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
      xmin  ymin        xmax        ymax  confidence  class         name
0  0.51638   0.0  152.912598  146.127411    0.275495      1  square root
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empt

       xmin      ymin        xmax        ymax  confidence  class  \
0  0.081551  0.601009  154.764099  107.848114    0.295794      3   

            name  
0  negative sign  
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
         xmin        ymin   xmax       ymax  confidence  class name
0  165.537415  188.201569  320.0  316.51889    0.319494      0    x
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty Dat

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
       xmin  ymin        xmax        ymax  confidence  class name
0  0.044777   0.0  153.996216  145.849091     0.47697      0    x
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Co

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, cla

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, cla

...

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, cla

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
       xmin        ymin        xmax        ymax  confidence  class       name
1  1.065849  174.124420  154.385284  318.287170    0.399623      9  summation
0  1.193100    0.634216  153.404984  147.087112    0.460305      9  summation
E

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
       xmin     ymin        xmax        ymax  confidence  class       name
0  0.627609  0.42585  152.297424  137.658752    0.697614      9  summation
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
       xmin      ymin        xmax        ymax  confidence  class       name
0  1.097481  1.083565  147.723297  129.861542    0.255952      9  summation
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
         xmin      ymin   xmax       ymax  confidence  class     name
0  160.994934  1.284382  320.0  8

Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
       xmin        ymin        xmax        ymax  confidence  class       name
0  0.000000    0.000000  152.905716  146.723663    0.688151      9  summation
1  0.858932  169.677002  149.688416  318.589539    0.420694      9  summation
         xmin        ymin        xmax        ymax  confidence  class  \
1    0.954674    0.000000  148.455536  149.227173    0.446721      9   
2    1.283325  173.894531  151.650055  318.461395    0.269963      9   
0  168.301636  180.827728  318.775116  319.712494    0.528081      2   

        name  
1  summation  
2  summation  
0  plus sign  
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
       xmin        ymin        xmax        ymax  confidence  class       name

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  